In [1]:
import requests
import csv
import json

url = "http://api.coincap.io/v2/assets/bitcoin/history?interval=d1&start=1592585794000&end=1613753794000"

payload = {}
headers= {}

response = requests.request("GET", url, headers=headers, data = payload)

json_data = json.loads(response.text.encode('utf8'))

In [2]:
bitcoin_data = json_data["data"]

In [3]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName("Stock Pyspark Example").getOrCreate()

In [4]:
df = spark.createDataFrame(bitcoin_data)

In [5]:
df.show(5,truncate=False)

+------------------------+---------------------+-------------+
|date                    |priceUsd             |time         |
+------------------------+---------------------+-------------+
|2020-06-20T00:00:00.000Z|9312.2640975682276296|1592611200000|
|2020-06-21T00:00:00.000Z|9360.6954642839445347|1592697600000|
|2020-06-22T00:00:00.000Z|9485.7188617077539610|1592784000000|
|2020-06-23T00:00:00.000Z|9643.1446484795214109|1592870400000|
|2020-06-24T00:00:00.000Z|9455.9143261593911834|1592956800000|
+------------------------+---------------------+-------------+
only showing top 5 rows

